In [3]:
import pandas as pd

In [4]:
df_emails = pd.read_csv('/Users/riadanas/Desktop/cybersecurity job project/Phishing_Email.csv')

In [5]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows

In [6]:
print(df_emails.shape)
df_emails.head(10)

(18650, 3)


,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email
5,5,global risk management operations sally congra...,Safe Email
6,6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email
7,7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email
8,8,"we owe you lots of money dear applicant , afte...",Phishing Email
9,9,re : coastal deal - with exxon participation u...,Safe Email


In [7]:
df_emails['Email Type'].value_counts()

Email Type
Safe Email        11322
Phishing Email     7328
Name: count, dtype: int64

In [8]:
df_emails['Email Text'].isna().sum()

np.int64(16)

# Data cleaning

In [20]:
df_emails.head(1)

,text,label,word_count,clean_text
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,230,disc uniformitarianism sex lang dick hudson ob...


In [9]:
# Drop the index column
df_emails = df_emails.drop(columns=['Unnamed: 0'])

# Drop rows with missing 'Email Text'
df_emails = df_emails.dropna(subset=['Email Text'])

# Standardize column names
df_emails.columns = ['text', 'label']

# Check updated shape and label distribution
df_emails.shape, df_emails['label'].value_counts()

((18634, 2),
 label
 Safe Email        11322
 Phishing Email     7312
 Name: count, dtype: int64)

In [10]:
df_emails.head(3)


,text,label
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email


# EDA

In [11]:
# Add word count column
df_emails['word_count'] = df_emails['text'].apply(lambda x: len(x.split()))

# Average word count
print(df_emails.groupby('label')['word_count'].mean().round(2))


label
Phishing Email    301.85
Safe Email        685.87
Name: word_count, dtype: float64


- Safe Emails tend to be twice as long as Phishing.

In [12]:
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Run this once to download stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Tokenization function
def tokenize_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())  # Lowercase, remove punctuation
    words = text.split()
    return [word for word in words if word not in stop_words]

# Apply to both phishing and safe emails
phishing_words = df_emails[df_emails['label'] == 'Phishing Email']['text'].apply(tokenize_text)
safe_words = df_emails[df_emails['label'] == 'Safe Email']['text'].apply(tokenize_text)

# Flatten lists
phishing_flat = [word for sublist in phishing_words for word in sublist]
safe_flat = [word for sublist in safe_words for word in sublist]

# Get most common 20 words
phishing_common = Counter(phishing_flat).most_common(20)
safe_common = Counter(safe_flat).most_common(20)

print("Phishing Common Words:\n", phishing_common)
print("\nSafe Email Common Words:\n", safe_common)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/riadanas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Phishing Common Words:
 [('email', 6618), ('free', 4329), ('com', 3976), ('get', 3871), ('company', 3863), ('information', 3846), ('please', 3827), ('money', 3794), ('business', 3575), ('one', 3572), ('us', 3544), ('time', 3194), ('e', 3154), ('report', 3122), ('http', 3120), ('new', 3032), ('click', 2938), ('make', 2739), ('order', 2687), ('mail', 2588)]

Safe Email Common Words:
 [('enron', 19271), ('university', 17181), ('language', 16843), ('one', 11386), ('subject', 11042), ('ect', 11005), ('would', 10834), ('information', 10817), ('email', 10671), ('new', 10338), ('please', 9925), ('linguistics', 8908), ('e', 8708), ('also', 8440), ('conference', 8019), ('may', 7704), ('de', 7468), ('com', 7063), ('papers', 6891), ('like', 6878)]


- We can spot here a few interesting keywords for Phishing:
- free, money, click, order

# Text processing

In [13]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    return ' '.join([word for word in text.split() if word not in stop_words])

df_emails['clean_text'] = df_emails['text'].apply(clean_text)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df_emails['clean_text'])

y = df_emails['label'].apply(lambda x: 1 if x == 'Phishing Email' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)


(14907, 3000) (3727, 3000)


In [14]:
df_emails.head(10)

,text,label,word_count,clean_text
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,230,disc uniformitarianism sex lang dick hudson ob...
1,the other side of * galicismos * * galicismo *...,Safe Email,91,side galicismos galicismo spanish term names i...
2,re : equistar deal tickets are you still avail...,Safe Email,305,equistar deal tickets still available assist r...
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,96,hello hot lil horny toy one dream open minded ...
4,software at incredibly low prices ( 86 % lower...,Phishing Email,91,software incredibly low prices lower drapery s...
5,global risk management operations sally congra...,Safe Email,592,global risk management operations sally congra...
6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email,153,sun aug wintermute mentioned impression get re...
7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email,1384,entourage stockmogul newsletter ralph velez ge...
8,"we owe you lots of money dear applicant , afte...",Phishing Email,142,owe lots money dear applicant review upon rece...
9,re : coastal deal - with exxon participation u...,Safe Email,359,coastal deal exxon participation project agree...


# Modeling and Eval

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Model Training
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Safe", "Phishing"]))

Confusion Matrix:
 [[2121   88]
 [  58 1460]]

Classification Report:
               precision    recall  f1-score   support

        Safe       0.97      0.96      0.97      2209
    Phishing       0.94      0.96      0.95      1518

    accuracy                           0.96      3727
   macro avg       0.96      0.96      0.96      3727
weighted avg       0.96      0.96      0.96      3727



In [16]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

def objective(trial):
    # Suggest hyperparameters
    c = trial.suggest_loguniform('C', 1e-3, 1e2)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = 'liblinear' if penalty == 'l1' else 'lbfgs'

    # Create model
    model = LogisticRegression(C=c, penalty=penalty, solver=solver, max_iter=1000)

    # Cross-validation F1 score
    score = cross_val_score(model, X_train, y_train, scoring='f1', cv=3).mean()
    return score

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Best params
print("Best params:", study.best_params)


/Users/riadanas/Desktop/cybersecurity job project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-18 11:43:54,323] A new study created in memory with name: no-name-fa57556f-8966-4ee1-a5fd-d0009fc0881a
/var/folders/1h/14p077t95lbcy6x5tc6bqx3h0000gn/T/ipykernel_80590/268487825.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c = trial.suggest_loguniform('C', 1e-3, 1e2)
[I 2025-06-18 11:43:54,510] Trial 0 finished with value: 0.9549352923366515 and parameters: {'C': 4.708105935899912, 'penalty': 'l2'}. Best is trial 0 with value: 0.9549352923366515.
/var/folders/1h/14p077t95lbcy6x5tc6bqx3h0000gn/T/ipykernel_80590/268487825.py:8:

Best params: {'C': 14.064080009107023, 'penalty': 'l2'}


In [17]:
best_params = study.best_params
final_model = LogisticRegression(**best_params, max_iter=1000)
final_model.fit(X_train, y_train)
final_preds = final_model.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, final_preds))
print("\nClassification Report:\n", classification_report(y_test, final_preds, target_names=["Safe", "Phishing"]))


Confusion Matrix:
 [[2118   91]
 [  32 1486]]

Classification Report:
               precision    recall  f1-score   support

        Safe       0.99      0.96      0.97      2209
    Phishing       0.94      0.98      0.96      1518

    accuracy                           0.97      3727
   macro avg       0.96      0.97      0.97      3727
weighted avg       0.97      0.97      0.97      3727



- RECALL MATTERS MOST: 
- In phishing detection, missing a phishing email (false negative) is much worse than wrongly flagging a legit email (false positive).

# Testing

In [18]:
# Sample emails
sample_emails = [
    # 1 - Obvious phishing
    "Your PayPal account has been suspended. Click the link below to restore access immediately.",
    
    # 2 - Safe
    "Hi John, just confirming our lunch meeting next Tuesday at 1pm. Let me know if that still works.",
    
    # 3 - Subtle phishing
    "This is IT Support. Please upload your password file to the link below so we can verify access before the upgrade.",
    
    # 4 - Safe
    "Reminder: Your HR policy document is due for review. Please acknowledge by Friday.",
    
    # 5 - Obvious phishing
    "You’ve won a £1000 Amazon voucher! Click here to claim your reward before it expires!",
    
    # 6 - Safe
    "Team, the weekly report is attached. Let's review it briefly before the 10:30 meeting.",
    
    # 7 - Subtle phishing
    "Please update your payroll info to avoid delays. Use the secure form here: update-payroll-data.co",
    
    # 8 - Safe
    "Good morning, attached is the updated invoice for March. Let me know if you have any questions.",
    
    # 9 - Obvious phishing
    "Final warning: Your email access will be revoked. Click now to confirm your credentials.",
    
    # 10 - Subtle phishing
    "Dear user, due to unusual activity, we've restricted your access. Use the form to reinstate your email privileges."
]

# Clean them using the same clean_text() function
sample_cleaned = [clean_text(email) for email in sample_emails]

# Vectorize using the existing TF-IDF model
sample_vectors = vectorizer.transform(sample_cleaned)

# Predict using the trained logistic regression model
sample_preds = model.predict(sample_vectors)
sample_probs = model.predict_proba(sample_vectors)

# Display results
for text, pred, prob in zip(sample_emails, sample_preds, sample_probs):
    label = "Phishing" if pred == 1 else "Safe"
    confidence = round(max(prob), 2)
    print(f"\nEmail:\n{text}\n→ Prediction: {label} (Confidence: {confidence})")



Email:
Your PayPal account has been suspended. Click the link below to restore access immediately.
→ Prediction: Phishing (Confidence: 0.97)

Email:
Hi John, just confirming our lunch meeting next Tuesday at 1pm. Let me know if that still works.
→ Prediction: Safe (Confidence: 0.97)

Email:
This is IT Support. Please upload your password file to the link below so we can verify access before the upgrade.
→ Prediction: Safe (Confidence: 0.77)

Email:
Reminder: Your HR policy document is due for review. Please acknowledge by Friday.
→ Prediction: Safe (Confidence: 0.9)

Email:
You’ve won a £1000 Amazon voucher! Click here to claim your reward before it expires!
→ Prediction: Phishing (Confidence: 0.93)

Email:
Team, the weekly report is attached. Let's review it briefly before the 10:30 meeting.
→ Prediction: Safe (Confidence: 0.96)

Email:
Please update your payroll info to avoid delays. Use the secure form here: update-payroll-data.co
→ Prediction: Phishing (Confidence: 0.58)

Email:
G